In [ ]:
coords = {} # node id is the key
adjlist = {} # node id is the key
with open('input.txt','r') as f:
    V = int(f.readline()) 
    for i in range(V):
        strs = f.readline().split()
        nid, x, y = strs[0], int(strs[1]), int(strs[2])
        coords[nid] = (x, y) # x, y kept as a tuple 
        adjlist[nid] = [] #create empty list for each node's adjnodes

    E = int(f.readline())
    for i in range(E):
        strs = f.readline().split()
        n1, n2, c = strs[0], strs[1], int(strs[2])
        adjlist[n1].append((n2, c)) # (n2, c) tuple
    startnid = f.readline().rstrip()
    goalnid = f.readline().rstrip()

print('graph')
for nid in adjlist:
    print(nid, coords[nid], '--->',  adjlist[nid])
    for tup in adjlist[nid]:
        print('\t', tup[0], tup[1])
print('start', startnid, 'goal', goalnid)

In [ ]:
def heuristic(nid: str, goal: str, coords: dict) -> float:
    """Return straight-line (Euclidean) distance between current node nid and the goal.

    This is admissible (never overestimates) if actual edge costs are >= straight line.
    Inputs:
        nid  : id of current node
        goal : id of goal node
        coords: mapping node_id -> (x,y)
    Output:
        float distance value (h(n))
    """
    (x1, y1) = coords[nid]
    (xg, yg) = coords[goal]
    return math.hypot(x1 - xg, y1 - yg)

# Quick sanity check of heuristic values (optional, can comment out later)
for n in coords:
    print(f"h({n}) = {heuristic(n, goalnid, coords):.2f}")

In [10]:
class State:
    """Represents a node in the A* search frontier/closed set.

    Attributes:
        nid (str): Node id (graph vertex label)
        g (float): Cost from start to this node (g(n))
        h (float): Heuristic estimate from this node to goal (h(n))
        f (float): Estimated total cost (f(n)=g+h)
        parent (State|None): Pointer to parent State for path reconstruction
    """
    __slots__ = ("nid", "g", "h", "f", "parent")

    def __init__(self, nid: str, g: float, h: float, parent: 'State' = None):
        self.nid = nid
        self.g = g
        self.h = h
        self.f = g + h
        self.parent = parent

    def __lt__(self, other: 'State'):
        # PriorityQueue uses < to break ties; prefer smaller f, then larger g (optional heuristic)
        if self.f == other.f:
            return self.g > other.g
        return self.f < other.f

    def __repr__(self):
        return f"State({self.nid}, g={self.g}, h={self.h:.2f}, f={self.f:.2f})"

    def path(self):
        """Reconstruct path from start to this node (returns list of node ids)."""
        out = []
        cur = self
        while cur:
            out.append(cur.nid)
            cur = cur.parent
        return list(reversed(out))

In [ ]:
from queue import PriorityQueue

# A* Search Implementation

def astar(start_id: str, goal_id: str, adjlist: dict, coords: dict):
    """Perform A* search and return (path, explored_count, goal_state).

    Steps:
        1. Initialize open priority queue with start state (g=0, h=h(start)).
        2. Maintain best_g dict: lowest discovered g for each node.
        3. Pop smallest f from open; if goal -> success.
        4. For each neighbor, compute tentative_g; if better, push/update.
        5. Use closed set to avoid re-expanding nodes with finalized best g.
    """
    openQ = PriorityQueue()
    h0 = heuristic(start_id, goal_id, coords)
    start_state = State(start_id, g=0, h=h0, parent=None)
    openQ.put(start_state)

    best_g = {start_id: 0}
    closed = set()
    explored = 0

    while not openQ.empty():
        current: State = openQ.get()
        explored += 1
        print(f"POP -> {current}")  # Trace step

        if current.nid == goal_id:
            print("Goal reached! Reconstructing path...")
            return current.path(), explored, current

        if current.nid in closed:
            # Already processed with a better g
            print(f"Skip {current.nid} (already closed)")
            continue

        closed.add(current.nid)

        for (nbr, cost) in adjlist[current.nid]:
            tentative_g = current.g + cost
            # If this path to neighbor is better (lower g) than any previous one
            if nbr not in best_g or tentative_g < best_g[nbr]:
                h = heuristic(nbr, goal_id, coords)
                nbr_state = State(nbr, tentative_g, h, parent=current)
                best_g[nbr] = tentative_g
                openQ.put(nbr_state)
                print(f"  PUSH -> {nbr_state} via {current.nid} (edge cost {cost})")
            else:
                print(f"  IGNORE neighbor {nbr} (tentative g={tentative_g} >= best {best_g[nbr]})")

    print("Failure: goal not reachable")
    return None, explored, None

# Run A* on provided graph
path, explored_count, goal_state = astar(startnid, goalnid, adjlist, coords)
print("\nRESULT")
print("Explored states:", explored_count)
print("Path:", path)
if goal_state:
    print("Total cost g(goal):", goal_state.g)

## A* Search Implementation Explanation
Line-by-line overview of the code in the next cell:
1. Import PriorityQueue for ordering by f(n).
2. Define `astar` function encapsulating the algorithm.
3. Compute `h0` = heuristic(start, goal) and create initial `State` (g=0, f=h0).
4. Initialize data structures:
   - `openQ`: frontier (min-heap by f)
   - `best_g`: maps node -> best discovered g
   - `closed`: nodes whose optimal g is finalized
   - `explored`: counter of pops
5. Main loop: while frontier not empty, pop smallest f state.
6. If popped node is goal: reconstruct path using parent links.
7. Skip node if already in `closed` (we found a better path earlier).
8. Otherwise add it to `closed` and expand neighbors.
9. For each neighbor (nbr, cost):
   - Compute `tentative_g = current.g + cost`.
   - If this g is better than any recorded (`nbr not in best_g or tentative_g < best_g[nbr]`):
     * Compute `h = heuristic(nbr, goal)`.
     * Create new `State` with updated g,h,f and parent.
     * Update `best_g` and push onto `openQ`.
   - Else ignore inferior path.
10. If frontier empties without reaching goal -> failure.
11. After run, print path, explored count, and total cost.

The trace prints POP / PUSH / IGNORE show the search dynamics. The heuristic used is Euclidean distance, which is admissible if edge costs are at least the straight-line distance lower bound (true for geometric graphs).